In [1]:
import sqlite3
import csv
import pandas as pd

# Connect to the database
conn = sqlite3.connect('company_xyz.db')
cursor = conn.cursor()

# Create the schema
cursor.execute('''
    CREATE TABLE Items (
        item_id INTEGER PRIMARY KEY,
        item_name TEXT
    )
''')

cursor.execute('''
    CREATE TABLE Customer (
        customer_id INTEGER PRIMARY KEY,
        age INTEGER
    )
''')

cursor.execute('''
    CREATE TABLE Sales (
        sales_id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        FOREIGN KEY (customer_id) REFERENCES Customer(customer_id)
    )
''')

cursor.execute('''
    CREATE TABLE Orders (
        order_id INTEGER PRIMARY KEY,
        sales_id INTEGER,
        item_id INTEGER,
        quantity INTEGER,
        FOREIGN KEY (sales_id) REFERENCES Sales(sales_id),
        FOREIGN KEY (item_id) REFERENCES Items(item_id)
       )
''')

# Insert sample data
cursor.executemany('''
    INSERT INTO Items (item_name) VALUES (?)
''', [('x',), ('y',), ('z',)])

cursor.executemany('''
    INSERT INTO Customer (age) VALUES (?)
''', [(23,), (34,), (27,), (59,), (17,)])

cursor.executemany('''
    INSERT INTO Sales (customer_id) VALUES (?)
''', [(1,), (2,), (3,), (4,), (5,), (2,), (3,), (4,), (3,), (4,), (5,), (4,), (5,), (5,)])

cursor.executemany('''
    INSERT INTO Orders (sales_id, item_id, quantity) VALUES (?, ?, ?)
''', [(1, 1, 0), (1, 2, 15), (1, 3, 20),
      (2, 1, 12), (1, 3, 18),
      (3, 2, 15), (3, 3, 22),
      (4, 1, 8), (4, 2, 12), (4, 3, 18),
      (5, 2, 9), (5, 3, 15),
      (6, 1, 10), (6, 2, 15),
      (7, 1, 9), (7, 3, 14),
      (8, 2, 8), (8, 3, 12),
      (9, 1, 7), (9, 2, 11), (9, 3, 16),
      (10, 1, 5), (10, 2, 8), (10, 3, 11)])

# Commit the changes
conn.commit()



In [2]:
# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])

# Print content of each table
for table in tables:
    print("\n",table[0])
    ndf = pd.read_sql_query(f"SELECT * FROM {table[0]};",conn)
    print(ndf)



Tables in the database:
Items
Customer
Sales
Orders

 Items
   item_id item_name
0        1         x
1        2         y
2        3         z

 Customer
   customer_id  age
0            1   23
1            2   34
2            3   27
3            4   59
4            5   17

 Sales
    sales_id  customer_id
0          1            1
1          2            2
2          3            3
3          4            4
4          5            5
5          6            2
6          7            3
7          8            4
8          9            3
9         10            4
10        11            5
11        12            4
12        13            5
13        14            5

 Orders
    order_id  sales_id  item_id  quantity
0          1         1        1         0
1          2         1        2        15
2          3         1        3        20
3          4         2        1        12
4          5         1        3        18
5          6         3        2        15
6          7         3  

**Approach using Purely SQL**

In [3]:
# Define the SQL query
query = """
SELECT c.customer_id, i.item_id, SUM(o.quantity) as total_quantity
FROM Orders o
JOIN Sales s ON o.sales_id = s.sales_id
JOIN Customer c ON s.customer_id = c.customer_id
JOIN Items i ON o.item_id = i.item_id
WHERE c.age BETWEEN 18 AND 35 AND o.quantity IS NOT NULL
GROUP BY c.customer_id, i.item_id
HAVING total_quantity > 0
"""

# Execute the SQL query
cursor.execute(query)

# Save the results to a CSV file
with open('output_purely_SQL.csv', 'w', newline='') as cf:
    writer = csv.writer(cf, delimiter=';')
    writer.writerow(['customer_id', 'item_id', 'total_quantity'])
    for row in cursor.fetchall():
        writer.writerow(row)

# Commit the changes
conn.commit()

**Approach using Pandas**

In [4]:
# Load the data from the Orders and Customer tables into Pandas dataframes
orders = pd.read_sql_query("""
SELECT c.customer_id, o.item_id, o.quantity, c.age
FROM Orders o
JOIN Sales s ON o.sales_id = s.sales_id
JOIN Customer c ON s.customer_id = c.customer_id
""", conn)

# Filter the dataframe to only include customers aged 18-35
filtered = orders[(orders['age'] >= 18) & (orders['age'] <= 35)]

# Group the data by customer and item, and calculate the total quantity purchased for each group
grouped = filtered.groupby(['customer_id', 'item_id']).sum().reset_index()

# Filter out groups with a total quantity of 0
filter_grouped = grouped[grouped['quantity'] > 0]

# Filtered and final output without age column
filtered_grouped= filter_grouped[['customer_id','item_id','quantity']]

# Save the pivoted dataframe to a CSV file with a semicolon delimiter
filtered_grouped.to_csv('output_pandas.csv', sep=';', index=False)

# Commit the changes
conn.commit()

# Close the connection
conn.close()